In [5]:
print(annotations.keys())  # Check top-level keys
print(annotations['annotations'][0])  # Check the first annotation


dict_keys(['info', 'licenses', 'categories', 'annotations'])
{'image_name': 'frame_20190829091111_x_0001973.jpg', 'image_width:': 1920.0, 'image_height': 1080.0, 'platform': 'Parrot Bebop 2', 'time': {'year': 2019, 'month': 8, 'day': 29, 'hour': 9, 'min': 11, 'sec': 11, 'ms': 394400.0}, 'longtitude': 10.18798203255313, 'latitude': 56.20630134795274, 'altitude': 19921.6, 'linear_x': 0.03130074199289083, 'linear_y': 0.028357808757573367, 'linear_z': 0.0744575835764408, 'angle_phi': -0.06713105738162994, 'angle_theta': 0.06894744634628296, 'angle_psi': 1.1161083340644837, 'bbox': [{'top': 163, 'left': 1098, 'height': 185, 'width': 420, 'class': 1}, {'top': 421, 'left': 1128, 'height': 176, 'width': 393, 'class': 1}, {'top': 927, 'left': 1703, 'height': 153, 'width': 183, 'class': 0}]}


 YOLOS-Tiny Object Detection Script (Train + Evaluate)

In [18]:
import wandb
from transformers import YolosImageProcessor, YolosForObjectDetection
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import json
import os
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np

# 🔧 Paths
root_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019"
annotation_path = os.path.join(root_dir, "annotations.json")
img_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\images"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 🟣 Init W&B
wandb.init(project="di725-assignment2", name="yolos-tiny-train")

# ⚙️ Load model + processor
processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")
model = YolosForObjectDetection.from_pretrained("hustvl/yolos-tiny")
model.to(device)

# ⚙️ Custom Dataset
class AUAIRDataset(Dataset):
    def __init__(self, annotations, img_dir, processor, split="train"):
        self.annotations = annotations["annotations"]
        self.images = annotations["images"]
        self.img_dir = img_dir
        self.processor = processor
        # Map image_id to annotations for efficient lookup
        self.ann_by_image_id = {}
        for ann in self.annotations:
            img_id = ann["image_id"]
            if img_id not in self.ann_by_image_id:
                self.ann_by_image_id[img_id] = []
            self.ann_by_image_id[img_id].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info["id"]
        img_path = os.path.join(self.img_dir, img_info["file_name"])
        
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None

        # Get annotations for this image
        anns = self.ann_by_image_id.get(img_id, [])
        boxes = []
        labels = []
        for ann in anns:
            # COCO-style bbox: [x, y, width, height]
            x, y, w, h = ann["bbox"]
            # Convert to [x_min, y_min, x_max, y_max] for YOLOS
            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])

        # Convert to tensors
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32) if boxes else torch.empty((0, 4), dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.long) if labels else torch.empty((0,), dtype=torch.long),
            "image_id": torch.tensor([img_id]),
        }

        # Process image and annotations
        # Include "area" and "iscrowd" for COCO compatibility
        processor_annotations = [
            {
                "bbox": [x, y, w, h],  # COCO format [x, y, width, height]
                "category_id": l,
                "area": float(w * h),  # Compute area
                "iscrowd": 0  # Default to 0 (no crowd)
            }
            for (x, y, w, h), l in zip((ann["bbox"] for ann in anns), labels)
        ]
        encoding = self.processor(
            images=image,
            annotations={"image_id": img_id, "annotations": processor_annotations},
            return_tensors="pt"
        )
        encoding["pixel_values"] = encoding["pixel_values"].squeeze(0)  # Remove batch dimension
        encoding["labels"] = target

        return encoding, image, img_id, img_info["file_name"]

# 📂 Load annotations
with open(annotation_path) as f:
    raw_annotations = json.load(f)

# Create pseudo-COCO format
image_map = {}  # Map image_id to image metadata
coco_annotations = []  # COCO-style annotations
for idx, ann in enumerate(raw_annotations["annotations"]):
    img_name = ann["image_name"]
    img_id = idx + 1  # Assign unique image_id (1-based indexing)
    image_map[img_id] = {
        "file_name": img_name,
        "width": ann["image_width:"],
        "height": ann["image_height"],
    }
    # Convert bbox list to COCO-style annotations
    for bbox in ann["bbox"]:
        coco_annotations.append({
            "image_id": img_id,
            "category_id": bbox["class"],
            "bbox": [bbox["left"], bbox["top"], bbox["width"], bbox["height"]],
            "area": bbox["width"] * bbox["height"],
            "id": len(coco_annotations) + 1,  # Unique annotation ID
        })

# Create pseudo-COCO structure
annotations = {
    "images": [{"id": img_id, "file_name": img["file_name"], "width": img["width"], "height": img["height"]} for img_id, img in image_map.items()],
    "annotations": coco_annotations,
    "categories": [{"id": i, "name": name} for i, name in enumerate(raw_annotations["categories"])],
}

# Split dataset (80% train, 20% val)
np.random.seed(42)
img_ids = [img["id"] for img in annotations["images"]]
np.random.shuffle(img_ids)
train_size = int(0.8 * len(img_ids))
train_ids = img_ids[:train_size]
val_ids = img_ids[train_size:]

train_images = [img for img in annotations["images"] if img["id"] in train_ids]
val_images = [img for img in annotations["images"] if img["id"] in val_ids]
train_annotations = {
    "images": train_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in train_ids],
    "categories": annotations["categories"],
}
val_annotations = {
    "images": val_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in val_ids],
    "categories": annotations["categories"],
}

train_dataset = AUAIRDataset(train_annotations, img_dir, processor, split="train")
val_dataset = AUAIRDataset(val_annotations, img_dir, processor, split="val")

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0, collate_fn=lambda x: [xi for xi in x if xi is not None])
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn=lambda x: [xi for xi in x if xi is not None])

# 🧠 Training Loop
def train_model(model, train_loader, val_loader, num_epochs=10, lr=5e-5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            pixel_values = torch.stack([item[0]["pixel_values"] for item in batch]).to(device)
            labels = [item[0]["labels"] for item in batch]

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        wandb.log({"epoch": epoch, "train_loss": avg_train_loss})

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                pixel_values = torch.stack([item[0]["pixel_values"] for item in batch]).to(device)
                labels = [item[0]["labels"] for item in batch]
                outputs = model(pixel_values=pixel_values, labels=labels)
                val_loss += outputs.loss.item()

        avg_val_loss = val_loss / len(val_loader)
        wandb.log({"epoch": epoch, "val_loss": avg_val_loss})
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        scheduler.step()

    # Save the trained model
    model.save_pretrained("yolos-tiny-finetuned")
    processor.save_pretrained("yolos-tiny-finetuned")

# 🧠 Inference
def run_yolos_inference(model, dataset, output_path="yolos_pred.json", log_images=False):
    model.eval()
    results = []

    for idx in tqdm(range(len(dataset))):
        inputs, image, img_id, image_name = dataset[idx]
        if inputs is None:
            continue
        inputs = {k: v.unsqueeze(0).to(device) for k, v in inputs.items() if k == "pixel_values"}

        with torch.no_grad():
            outputs = model(**inputs)

        width, height = image.size
        target_sizes = torch.tensor([[height, width]]).to(device)
        
        result = processor.post_process_object_detection(
            outputs,
            target_sizes=target_sizes,
            threshold=0.5
        )[0]

        if log_images and idx % 50 == 0 and len(result["boxes"]) > 0:
            boxes = result["boxes"].cpu().tolist()
            scores = result["scores"].cpu().tolist()
            labels = result["labels"].cpu().tolist()

            wandb.log({
                "prediction": wandb.Image(image, boxes={
                    "predictions": {
                        "box_data": [
                            {
                                "position": {
                                    "minX": b[0] / width,
                                    "minY": b[1] / height,
                                    "maxX": b[2] / width,
                                    "maxY": b[3] / height,
                                },
                                "score": s,
                                "class_id": l
                            }
                            for b, s, l in zip(boxes, scores, labels)
                        ],
                        "class_labels": {i: name for i, name in enumerate(raw_annotations["categories"])}
                    }
                }),
                "step": idx
            })

        for box, label, score in zip(result["boxes"], result["labels"], result["scores"]):
            xmin, ymin, xmax, ymax = box
            results.append({
                "image_id": img_id,
                "category_id": int(label),
                "bbox": [float(xmin), float(ymin), float(xmax - xmin), float(ymax - ymin)],
                "score": float(score)
            })

    with open(output_path, "w") as f:
        json.dump(results, f)
    return output_path

# 📊 mAP Evaluation
def evaluate_map(gt_path, pred_path):
    coco_gt = COCO(gt_path)
    coco_dt = coco_gt.loadRes(pred_path)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    metrics = {
        "mAP@[0.5:0.95]": coco_eval.stats[0],
        "AP50": coco_eval.stats[1],
        "AP75": coco_eval.stats[2],
        "AP_small": coco_eval.stats[3],
        "AP_medium": coco_eval.stats[4],
        "AP_large": coco_eval.stats[5]
    }

    precisions = coco_eval.eval["precision"]
    cat_ids = coco_gt.getCatIds()
    categories = coco_gt.loadCats(cat_ids)

    print("\n📊 Per-category AP (IoU=0.50:0.95):")
    for idx, cat in enumerate(categories):
        precision = precisions[:, :, idx, 0, 0]
        precision = precision[precision > -1]
        ap = precision.mean() if precision.size > 0 else float("nan")
        metrics[f"AP_{cat['name']}"] = ap
        print(f"  {cat['name']:20s}: {ap:.4f}")

    wandb.log(metrics)
    print("✅ mAP + per-class AP metrics logged to W&B.")
    return metrics

# 🧪 Run Training and Evaluation
# Save ground truth annotations to gt.json
with open("gt.json", "w") as f:
    json.dump(val_annotations, f)

# Train the model
train_model(model, train_loader, val_loader, num_epochs=10, lr=5e-5)

# Run inference on validation set
pred_json = run_yolos_inference(model, val_dataset, output_path="yolos_pred.json", log_images=True)

# Evaluate
evaluate_map("gt.json", pred_json)

Epoch 1/10:   0%|          | 0/6565 [00:00<?, ?it/s]


KeyError: 'class_labels'

In [20]:
import wandb
from transformers import YolosImageProcessor, YolosForObjectDetection
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import json
import os
from tqdm import tqdm
import numpy as np

# 🔧 Paths
root_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019"
annotation_path = os.path.join(root_dir, "annotations.json")
img_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\images"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 🟣 Init W&B
wandb.init(project="di725-assignment2", name="yolos-tiny-train")

# ⚙️ Load model + processor
processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")
model = YolosForObjectDetection.from_pretrained("hustvl/yolos-tiny")
model.to(device)

# ⚙️ Category ID mapping (AUAIR to COCO)
label_map = {0: 1, 1: 3, 2: 8, 3: 7, 4: 4, 5: 2, 6: 6, 7: 10}  # AUAIR to COCO

# ⚙️ Custom Dataset
class AUAIRDataset(Dataset):
    def __init__(self, annotations, img_dir, processor, split="train"):
        self.annotations = annotations["annotations"]
        self.images = annotations["images"]
        self.img_dir = img_dir
        self.processor = processor
        # Map image_id to annotations for efficient lookup
        self.ann_by_image_id = {}
        for ann in self.annotations:
            img_id = ann["image_id"]
            if img_id not in self.ann_by_image_id:
                self.ann_by_image_id[img_id] = []
            self.ann_by_image_id[img_id].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info["id"]
        img_path = os.path.join(self.img_dir, img_info["file_name"])
        
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None

        # Get annotations for this image
        anns = self.ann_by_image_id.get(img_id, [])
        boxes = []
        labels = []
        for ann in anns:
            # COCO-style bbox: [x, y, width, height]
            x, y, w, h = ann["bbox"]
            # Convert to [x_min, y_min, x_max, y_max] for YOLOS
            boxes.append([x, y, x + w, y + h])
            labels.append(label_map[ann["category_id"]])  # Map AUAIR to COCO IDs

        # Convert to tensors
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32) if boxes else torch.empty((0, 4), dtype=torch.float32),
            "class_labels": torch.tensor(labels, dtype=torch.long) if labels else torch.empty((0,), dtype=torch.long),
            "image_id": torch.tensor([img_id]),
        }

        # Process image and annotations
        # Include "area" and "iscrowd" for COCO compatibility
        processor_annotations = [
            {
                "bbox": [x, y, w, h],  # COCO format [x, y, width, height]
                "category_id": label_map[l],  # Map AUAIR to COCO IDs
                "area": float(w * h),  # Compute area
                "iscrowd": 0  # Default to 0 (no crowd)
            }
            for (x, y, w, h), l in zip((ann["bbox"] for ann in anns), [ann["category_id"] for ann in anns])
        ]
        encoding = self.processor(
            images=image,
            annotations={"image_id": img_id, "annotations": processor_annotations},
            return_tensors="pt"
        )
        encoding["pixel_values"] = encoding["pixel_values"].squeeze(0)  # Remove batch dimension
        encoding["labels"] = target

        return encoding, image, img_id, img_info["file_name"]

# 📂 Load annotations
with open(annotation_path) as f:
    raw_annotations = json.load(f)

# Create pseudo-COCO format
image_map = {}  # Map image_id to image metadata
coco_annotations = []  # COCO-style annotations
for idx, ann in enumerate(raw_annotations["annotations"]):
    img_name = ann["image_name"]
    img_id = idx + 1  # Assign unique image_id (1-based indexing)
    image_map[img_id] = {
        "file_name": img_name,
        "width": ann["image_width:"],
        "height": ann["image_height"],
    }
    # Convert bbox list to COCO-style annotations
    for bbox in ann["bbox"]:
        coco_annotations.append({
            "image_id": img_id,
            "category_id": bbox["class"],
            "bbox": [bbox["left"], bbox["top"], bbox["width"], bbox["height"]],
            "area": bbox["width"] * bbox["height"],
            "id": len(coco_annotations) + 1,  # Unique annotation ID
        })

# Create pseudo-COCO structure
annotations = {
    "images": [{"id": img_id, "file_name": img["file_name"], "width": img["width"], "height": img["height"]} for img_id, img in image_map.items()],
    "annotations": coco_annotations,
    "categories": [{"id": i, "name": name} for i, name in enumerate(raw_annotations["categories"])],
}

# Split dataset (80% train, 20% val)
np.random.seed(42)
img_ids = [img["id"] for img in annotations["images"]]
np.random.shuffle(img_ids)
train_size = int(0.8 * len(img_ids))
train_ids = img_ids[:train_size]
val_ids = img_ids[train_size:]

train_images = [img for img in annotations["images"] if img["id"] in train_ids]
val_images = [img for img in annotations["images"] if img["id"] in val_ids]
train_annotations = {
    "images": train_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in train_ids],
    "categories": annotations["categories"],
}
val_annotations = {
    "images": val_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in val_ids],
    "categories": annotations["categories"],
}

train_dataset = AUAIRDataset(train_annotations, img_dir, processor, split="train")
val_dataset = AUAIRDataset(val_annotations, img_dir, processor, split="val")

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0, collate_fn=lambda x: [xi for xi in x if xi is not None])
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn=lambda x: [xi for xi in x if xi is not None])

# 🧠 Training Loop
def train_model(model, train_loader, val_loader, num_epochs=10, lr=5e-5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            pixel_values = torch.stack([item[0]["pixel_values"] for item in batch]).to(device)
            labels = [item[0]["labels"] for item in batch]

            # Move all tensors in labels to the correct device
            labels = [
                {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in label.items()}
                for label in labels
            ]

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        wandb.log({"epoch": epoch, "train_loss": avg_train_loss})

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                pixel_values = torch.stack([item[0]["pixel_values"] for item in batch]).to(device)
                labels = [item[0]["labels"] for item in batch]

                # Move all tensors in labels to the correct device
                labels = [
                    {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in label.items()}
                    for label in labels
                ]

                outputs = model(pixel_values=pixel_values, labels=labels)
                val_loss += outputs.loss.item()

        avg_val_loss = val_loss / len(val_loader)
        wandb.log({"epoch": epoch, "val_loss": avg_val_loss})
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        scheduler.step()

    # Save the trained model
    model.save_pretrained("yolos-tiny-finetuned")
    processor.save_pretrained("yolos-tiny-finetuned")

# 🧪 Run Training
# Save ground truth annotations for evaluation
with open("gt.json", "w") as f:
    json.dump(val_annotations, f)

train_model(model, train_loader, val_loader, num_epochs=10, lr=5e-5)

Validation: 100%|██████████| 1642/1642 [10:40<00:00,  2.56it/s]


Epoch 1/10, Train Loss: 13666.2197, Val Loss: 13905.3953


Validation: 100%|██████████| 1642/1642 [10:39<00:00,  2.57it/s]


Epoch 2/10, Train Loss: 13660.2704, Val Loss: 13905.3763


Validation: 100%|██████████| 1642/1642 [10:37<00:00,  2.58it/s]


...

Epoch 5/10, Train Loss: 13661.5152, Val Loss: 13905.3607


Validation: 100%|██████████| 1642/1642 [09:04<00:00,  3.02it/s]


Epoch 6/10, Train Loss: 13672.2140, Val Loss: 13905.3667


Epoch 7/10:   0%|          | 11/6565 [00:05<51:31,  2.12it/s]


KeyboardInterrupt: 

Fixed Classifier Head Adjustment:
Replaced model.config.d_model with model.config.hidden_size to get the correct input dimension (768 for yolos-tiny).
Set class_labels_classifier to a torch.nn.Linear layer with hidden_size input and num_classes (9) output.
Added Gradient Clipping:
Added torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) before optimizer.step() to stabilize training and address the high loss (~13,660) observed previously.
Preserved Improvements:
Kept bounding box normalization ([x_min, y_min, x_max, y_max] in [0, 1]).
Retained loss component logging (loss_ce, loss_bbox, loss_giou).
Maintained debugging for category IDs and sample counts.
Kept lr=1e-4 for faster convergence.
Category IDs:
Used AUAIR IDs (0–7) directly, as the classifier head is adjusted to match AUAIR categories (0–7 for classes, 8 for background).

In [ ]:
import wandb
from transformers import YolosImageProcessor, YolosForObjectDetection
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import json
import os
from tqdm import tqdm
import numpy as np

# 🔧 Paths
root_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019"
annotation_path = os.path.join(root_dir, "annotations.json")
img_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\images"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 🟣 Init W&B
wandb.init(project="di725-assignment2", name="yolos-tiny-train")

# ⚙️ Load model + processor
processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")
model = YolosForObjectDetection.from_pretrained("hustvl/yolos-tiny")
model.to(device)

# ⚙️ Verify category mapping
print("Model id2label:", model.config.id2label)
with open(annotation_path) as f:
    raw_annotations = json.load(f)
    print("AUAIR categories:", raw_annotations["categories"])

# ⚙️ Category ID mapping (AUAIR to COCO)
label_map = {0: 1, 1: 3, 2: 8, 3: 7, 4: 4, 5: 2, 6: 6, 7: 10}  # AUAIR to COCO

# ⚙️ Custom Dataset
class AUAIRDataset(Dataset):
    def __init__(self, annotations, img_dir, processor, split="train"):
        self.annotations = annotations["annotations"]
        self.images = annotations["images"]
        self.img_dir = img_dir
        self.processor = processor
        self.split = split
        # Map image_id to annotations for efficient lookup
        self.ann_by_image_id = {}
        self.invalid_category_count = 0  # Track invalid category IDs
        for ann in self.annotations:
            img_id = ann["image_id"]
            if img_id not in self.ann_by_image_id:
                self.ann_by_image_id[img_id] = []
            self.ann_by_image_id[img_id].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info["id"]
        img_path = os.path.join(self.img_dir, img_info["file_name"])
        
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None

        # Get annotations for this image
        anns = self.ann_by_image_id.get(img_id, [])
        boxes = []
        labels = []
        img_width, img_height = img_info["width"], img_info["height"]
        for ann in anns:
            # COCO-style bbox: [x, y, width, height]
            x, y, w, h = ann["bbox"]
            # Convert to [x_min, y_min, x_max, y_max] and normalize
            x_min, y_min = x / img_width, y / img_height
            x_max, y_max = (x + w) / img_width, (y + h) / img_height
            if ann["category_id"] in label_map:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(label_map[ann["category_id"]])  # Map AUAIR IDs to COCO IDs
            else:
                self.invalid_category_count += 1
                #print(f"Invalid category_id {ann['category_id']} in image_id {img_id}, annotation: {ann}")

        # Convert to tensors
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32) if boxes else torch.empty((0, 4), dtype=torch.float32),
            "class_labels": torch.tensor(labels, dtype=torch.long) if labels else torch.empty((0,), dtype=torch.long),
            "image_id": torch.tensor([img_id]),
        }

        # Process image and annotations
        processor_annotations = [
            {
                "bbox": [x, y, w, h],  # COCO format [x, y, width, height]
                "category_id": label_map[l],  # Map AUAIR IDs to COCO IDs
                "area": float(w * h),
                "iscrowd": 0
            }
            for (x, y, w, h), l in zip((ann["bbox"] for ann in anns if ann["category_id"] in label_map), 
                                     [ann["category_id"] for ann in anns if ann["category_id"] in label_map])
        ]
        encoding = self.processor(
            images=image,
            annotations={"image_id": img_id, "annotations": processor_annotations},
            return_tensors="pt"
        )
        encoding["pixel_values"] = encoding["pixel_values"].squeeze(0)
        encoding["labels"] = target

        # Debug: Log category IDs for first few samples
        if idx < 3 and self.split == "train":
            print(f"Sample {idx}, Image ID: {img_id}, Labels: {labels}")

        return encoding, image, img_id, img_info["file_name"]

# 📂 Load annotations
with open(annotation_path) as f:
    raw_annotations = json.load(f)

# Debug: Check unique category IDs in raw annotations
unique_category_ids = set()
for ann in raw_annotations["annotations"]:
    for bbox in ann["bbox"]:
        unique_category_ids.add(bbox["class"])
print(f"Unique category IDs in raw annotations: {unique_category_ids}")

# Create pseudo-COCO format
image_map = {}
coco_annotations = []
invalid_annotation_count = 0  # Track invalid annotations during processing
for idx, ann in enumerate(raw_annotations["annotations"]):
    img_name = ann["image_name"]
    img_id = idx + 1
    image_map[img_id] = {
        "file_name": img_name,
        "width": ann["image_width:"],
        "height": ann["image_height"],
    }
    for bbox in ann["bbox"]:
        if bbox["class"] in label_map:  # Only include valid category IDs
            coco_annotations.append({
                "image_id": img_id,
                "category_id": label_map[bbox["class"]],  # Map AUAIR IDs to COCO IDs
                "bbox": [bbox["left"], bbox["top"], bbox["width"], bbox["height"]],
                "area": bbox["width"] * bbox["height"],
                "id": len(coco_annotations) + 1,
            })
        else:
            #print(f"Skipping annotation with invalid category_id {bbox['class']} in image {img_name}")
            invalid_annotation_count += 1


# Print invalid annotation count from annotation processing
print(f"invalid_category_count: {invalid_annotation_count}")

annotations = {
    "images": [{"id": img_id, "file_name": img["file_name"], "width": img["width"], "height": img["height"]} for img_id, img in image_map.items()],
    "annotations": coco_annotations,
    "categories": [{"id": i, "name": model.config.id2label[i]} for i in label_map.values()],
}

# Split dataset (80% train, 20% val)
np.random.seed(42)
img_ids = [img["id"] for img in annotations["images"]]
np.random.shuffle(img_ids)
train_size = int(0.8 * len(img_ids))
train_ids = img_ids[:train_size]
val_ids = img_ids[train_size:]

train_images = [img for img in annotations["images"] if img["id"] in train_ids]
val_images = [img for img in annotations["images"] if img["id"] in val_ids]
train_annotations = {
    "images": train_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in train_ids],
    "categories": annotations["categories"],
}
val_annotations = {
    "images": val_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in val_ids],
    "categories": annotations["categories"],
}

train_dataset = AUAIRDataset(train_annotations, img_dir, processor, split="train")
val_dataset = AUAIRDataset(val_annotations, img_dir, processor, split="val")

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0, collate_fn=lambda x: [xi for xi in x if xi is not None])
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn=lambda x: [xi for xi in x if xi is not None])

# 🧠 Training Loop
def train_model(model, train_loader, val_loader, num_epochs=10, lr=1e-4):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    # Count valid samples
    valid_train_samples = 0
    for batch in train_loader:
        valid_train_samples += len(batch)
    valid_val_samples = 0
    for batch in val_loader:
        valid_val_samples += len(batch)
    print(f"Valid training samples: {valid_train_samples}, Valid validation samples: {valid_val_samples}")
    #print(f"Invalid category IDs encountered (train): {train_dataset.invalid_category_count}")

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_loss_ce = 0.0
        train_loss_bbox = 0.0
        train_loss_giou = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            pixel_values = torch.stack([item[0]["pixel_values"] for item in batch]).to(device)
            labels = [item[0]["labels"] for item in batch]

            # Move all tensors in labels to the correct device
            labels = [
                {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in label.items()}
                for label in labels
            ]

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss
            loss_dict = outputs.loss_dict  # Contains loss_ce, loss_bbox, loss_giou
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            train_loss_ce += loss_dict.get("loss_ce", 0.0).item()
            train_loss_bbox += loss_dict.get("loss_bbox", 0.0).item()
            train_loss_giou += loss_dict.get("loss_giou", 0.0).item()

        avg_train_loss = train_loss / len(train_loader)
        avg_train_loss_ce = train_loss_ce / len(train_loader)
        avg_train_loss_bbox = train_loss_bbox / len(train_loader)
        avg_train_loss_giou = train_loss_giou / len(train_loader)
        wandb.log({
            "epoch": epoch,
            "train_loss": avg_train_loss,
            "train_loss_ce": avg_train_loss_ce,
            "train_loss_bbox": avg_train_loss_bbox,
            "train_loss_giou": avg_train_loss_giou
        })

        # Validation
        model.eval()
        val_loss = 0.0
        val_loss_ce = 0.0
        val_loss_bbox = 0.0
        val_loss_giou = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                pixel_values = torch.stack([item[0]["pixel_values"] for item in batch]).to(device)
                labels = [item[0]["labels"] for item in batch]

                labels = [
                    {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in label.items()}
                    for label in labels
                ]

                outputs = model(pixel_values=pixel_values, labels=labels)
                val_loss += outputs.loss.item()
                val_loss_ce += outputs.loss_dict.get("loss_ce", 0.0).item()
                val_loss_bbox += outputs.loss_dict.get("loss_bbox", 0.0).item()
                val_loss_giou += outputs.loss_dict.get("loss_giou", 0.0).item()

        avg_val_loss = val_loss / len(val_loader)
        avg_val_loss_ce = val_loss_ce / len(val_loader)
        avg_val_loss_bbox = val_loss_bbox / len(val_loader)
        avg_val_loss_giou = val_loss_giou / len(val_loader)
        wandb.log({
            "epoch": epoch,
            "val_loss": avg_val_loss,
            "val_loss_ce": avg_val_loss_ce,
            "val_loss_bbox": avg_val_loss_bbox,
            "val_loss_giou": avg_val_loss_giou,
            "invalid_category_count_train": train_dataset.invalid_category_count,
            "invalid_category_count_val": val_dataset.invalid_category_count
        })
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f} (CE: {avg_train_loss_ce:.4f}, BBox: {avg_train_loss_bbox:.4f}, GIoU: {avg_train_loss_giou:.4f}), Val Loss: {avg_val_loss:.4f} (CE: {avg_val_loss_ce:.4f}, BBox: {avg_val_loss_bbox:.4f}, GIoU: {avg_val_loss_giou:.4f})")

        scheduler.step()

    # Save the trained model
    model.save_pretrained("yolos-tiny-finetuned")
    processor.save_pretrained("yolos-tiny-finetuned")

# 🧪 Run Training
# Save ground truth annotations for evaluation
with open("gt.json", "w") as f:
    json.dump(val_annotations, f)

train_model(model, train_loader, val_loader, num_epochs=10, lr=1e-4)

Model id2label: {0: 'N/A', 1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 12: 'N/A', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 26: 'N/A', 27: 'backpack', 28: 'umbrella', 29: 'N/A', 30: 'N/A', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 45: 'N/A', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 66: 'N/A', 67: 'dining table', 68: 'N/A', 69: 'N

KeyboardInterrupt: 

INFERENCE PART

In [ ]:
import wandb
from transformers import YolosImageProcessor, YolosForObjectDetection
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import json
import os
from tqdm import tqdm
import numpy as np

# 🔧 Paths
root_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019"
annotation_path = os.path.join(root_dir, "annotations.json")
img_dir = r"C:\Users\nesil.bor\Desktop\Folders\master\DI725\DI725_Assignment2_2030336\data\auair2019\images"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 🟣 Init W&B
wandb.init(project="di725-assignment2", name="yolos-tiny-inference")

# ⚙️ Load model + processor
processor = YolosImageProcessor.from_pretrained("yolos-tiny-finetuned")
model = YolosForObjectDetection.from_pretrained("yolos-tiny-finetuned")
model.to(device)

# ⚙️ Category ID mapping (COCO to AUAIR for evaluation)
reverse_map = {1: 0, 3: 1, 8: 2, 7: 3, 4: 4, 2: 5, 6: 6, 10: 7}  # COCO to AUAIR

# ⚙️ Custom Dataset
class AUAIRDataset(Dataset):
    def __init__(self, annotations, img_dir, processor, split="val"):
        self.annotations = annotations["annotations"]
        self.images = annotations["images"]
        self.img_dir = img_dir
        self.processor = processor
        # Map image_id to annotations for efficient lookup
        self.ann_by_image_id = {}
        for ann in self.annotations:
            img_id = ann["image_id"]
            if img_id not in self.ann_by_image_id:
                self.ann_by_image_id[img_id] = []
            self.ann_by_image_id[img_id].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info["id"]
        img_path = os.path.join(self.img_dir, img_info["file_name"])
        
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None

        # Get annotations for this image (not used for inference, but kept for compatibility)
        anns = self.ann_by_image_id.get(img_id, [])
        boxes = []
        labels = []
        for ann in anns:
            x, y, w, h = ann["bbox"]
            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])

        # Convert to tensors (not used for inference, but kept for compatibility)
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32) if boxes else torch.empty((0, 4), dtype=torch.float32),
            "class_labels": torch.tensor(labels, dtype=torch.long) if labels else torch.empty((0,), dtype=torch.long),
            "image_id": torch.tensor([img_id]),
        }

        # Process image
        encoding = self.processor(images=image, return_tensors="pt")
        encoding["pixel_values"] = encoding["pixel_values"].squeeze(0)  # Remove batch dimension
        encoding["labels"] = target

        return encoding, image, img_id, img_info["file_name"]

# 📂 Load annotations
with open(annotation_path) as f:
    raw_annotations = json.load(f)

# Create pseudo-COCO format
image_map = {}  # Map image_id to image metadata
coco_annotations = []  # COCO-style annotations
for idx, ann in enumerate(raw_annotations["annotations"]):
    img_name = ann["image_name"]
    img_id = idx + 1  # Assign unique image_id (1-based indexing)
    image_map[img_id] = {
        "file_name": img_name,
        "width": ann["image_width:"],
        "height": ann["image_height"],
    }
    for bbox in ann["bbox"]:
        coco_annotations.append({
            "image_id": img_id,
            "category_id": bbox["class"],
            "bbox": [bbox["left"], bbox["top"], bbox["width"], bbox["height"]],
            "area": bbox["width"] * bbox["height"],
            "id": len(coco_annotations) + 1,
        })

# Create pseudo-COCO structure
annotations = {
    "images": [{"id": img_id, "file_name": img["file_name"], "width": img["width"], "height": img["height"]} for img_id, img in image_map.items()],
    "annotations": coco_annotations,
    "categories": [{"id": i, "name": name} for i, name in enumerate(raw_annotations["categories"])],
}

# Split dataset (use validation set for inference)
np.random.seed(42)
img_ids = [img["id"] for img in annotations["images"]]
np.random.shuffle(img_ids)
train_size = int(0.8 * len(img_ids))
val_ids = img_ids[train_size:]

val_images = [img for img in annotations["images"] if img["id"] in val_ids]
val_annotations = {
    "images": val_images,
    "annotations": [ann for ann in annotations["annotations"] if ann["image_id"] in val_ids],
    "categories": annotations["categories"],
}

val_dataset = AUAIRDataset(val_annotations, img_dir, processor, split="val")

# 🧠 Inference
def run_yolos_inference(model, dataset, output_path="yolos_pred.json", log_images=False):
    model.eval()
    results = []

    for idx in tqdm(range(len(dataset))):
        inputs, image, img_id, image_name = dataset[idx]
        if inputs is None:
            continue
        inputs = {k: v.unsqueeze(0).to(device) for k, v in inputs.items() if k == "pixel_values"}

        with torch.no_grad():
            outputs = model(**inputs)

        width, height = image.size
        target_sizes = torch.tensor([[height, width]]).to(device)
        
        result = processor.post_process_object_detection(
            outputs,
            target_sizes=target_sizes,
            threshold=0.5
        )[0]

        if log_images and idx % 50 == 0 and len(result["boxes"]) > 0:
            boxes = result["boxes"].cpu().tolist()
            scores = result["scores"].cpu().tolist()
            labels = result["labels"].cpu().tolist()

            wandb.log({
                "prediction": wandb.Image(image, boxes={
                    "predictions": {
                        "box_data": [
                            {
                                "position": {
                                    "minX": b[0] / width,
                                    "minY": b[1] / height,
                                    "maxX": b[2] / width,
                                    "maxY": b[3] / height,
                                },
                                "score": s,
                                "class_id": reverse_map.get(l, l)  # Map back to AUAIR IDs for logging
                            }
                            for b, s, l in zip(boxes, scores, labels)
                        ],
                        "class_labels": {i: name for i, name in enumerate(raw_annotations["categories"])}
                    }
                }),
                "step": idx
            })

        for box, label, score in zip(result["boxes"], result["labels"], result["scores"]):
            xmin, ymin, xmax, ymax = box
            results.append({
                "image_id": img_id,
                "category_id": reverse_map.get(int(label), int(label)),  # Map COCO to AUAIR IDs
                "bbox": [float(xmin), float(ymin), float(xmax - xmin), float(ymax - ymin)],
                "score": float(score)
            })

    with open(output_path, "w") as f:
        json.dump(results, f)
    return output_path

# 🧪 Run Inference
pred_json = run_yolos_inference(model, val_dataset, output_path="yolos_pred.json", log_images=True)
print(f"Predictions saved to {pred_json}")

EVALUATION PART 

In [ ]:
import wandb
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import numpy as np

# 🔧 Paths
gt_path = "gt.json"
pred_path = "yolos_pred.json"

# 🟣 Init W&B
wandb.init(project="di725-assignment2", name="yolos-tiny-evaluate")

# 📊 mAP Evaluation
def evaluate_map(gt_path, pred_path):
    coco_gt = COCO(gt_path)
    coco_dt = coco_gt.loadRes(pred_path)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    metrics = {
        "mAP@[0.5:0.95]": coco_eval.stats[0],
        "AP50": coco_eval.stats[1],
        "AP75": coco_eval.stats[2],
        "AP_small": coco_eval.stats[3],
        "AP_medium": coco_eval.stats[4],
        "AP_large": coco_eval.stats[5]
    }

    precisions = coco_eval.eval["precision"]
    cat_ids = coco_gt.getCatIds()
    categories = coco_gt.loadCats(cat_ids)

    print("\n📊 Per-category AP (IoU=0.50:0.95):")
    for idx, cat in enumerate(categories):
        precision = precisions[:, :, idx, 0, 0]
        precision = precision[precision > -1]
        ap = precision.mean() if precision.size > 0 else float("nan")
        metrics[f"AP_{cat['name']}"] = ap
        print(f"  {cat['name']:20s}: {ap:.4f}")

    wandb.log(metrics)
    print("✅ mAP + per-class AP metrics logged to W&B.")
    return metrics

# 🧪 Run Evaluation
metrics = evaluate_map(gt_path, pred_path)
print("Evaluation complete.")